In [1]:
import pandas as pd
import re
import pathlib

In [5]:
def get_data():
  df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl')
  df_2 = df.astype(str, errors='ignore')
  df_2 = df_2.applymap(lambda x: x[1:] if x.startswith(".") else x)
  df_2 = df_2.applymap(lambda x: x.strip())
  filt = df_2.Ch.apply(len) < 5
  df_2 = df_2[~filt]
  return df, df_2

In [6]:
data, data_2 = get_data()

UnpicklingError: invalid load key, '\x1f'.

In [50]:
x = data[data['Lang_Ch'].str.contains("泰雅")]
x['Lang_Ch'].unique()

['泰雅_賽考利克', '泰雅_澤敖利', '泰雅_汶水', '泰雅_萬大', '泰雅_四季', '泰雅_宜蘭澤敖利']
Categories (43, object): ['卑南_初鹿', '卑南_南王', '卑南_建和', '卑南_知本', ..., '魯凱_東', '魯凱_茂林', '魯凱_萬山', '魯凱_霧台']

In [51]:
data_2.shape

(136385, 5)

In [80]:
r = data_2.copy()

def read_a_csv_and_drop_na(path):
    df = pd.read_csv(path, encoding="big5").dropna(how='all').dropna(axis=1, how='all')
    df['Lang_En'] = 'Atayal'
    df['Lang_Ch'] = '泰雅_賽考利克'
    df['From'] = "傳說故事精選篇"
    df = df.rename(columns={"Atayal": "Ab", "Mandarin": "Ch"})
    
    return df

for x in pathlib.Path("../atayal_data/").glob("*.csv"):
    c = read_a_csv_and_drop_na(x)
    r = pd.concat([r, c], ignore_index=True)
    


In [84]:
r.columns
r.drop('Unnamed: 2', inplace=True,axis=1)

KeyError: "['Unnamed: 2'] not found in axis"

In [85]:
r.to_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

In [87]:
df = pd.read_pickle('./Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")
df

,Lang_En,Lang_Ch,Ab,Ch,From
0,Sakizaya,撒奇萊雅,malalikid ku niyazu' i waluay a bulad.,八月份是部落的豐年祭。,詞典
1,Sakizaya,撒奇萊雅,kaudadan a demiad milalupela' kita.,下雨天我們一起去撿天使的眼淚。,詞典
2,Sakizaya,撒奇萊雅,i buyubuyu'an ku aadupen a mauzip.,動物生存在山裡。,詞典
3,Sakizaya,撒奇萊雅,u aam ku sakalanam tu sananal.,早餐吃的是稀飯。,詞典
4,Sakizaya,撒奇萊雅,aamen nu miaamay ku tubah ni Bunga!,乞丐去向Bunga乞討地瓜！,詞典
...,...,...,...,...,...
137190,Atayal,泰雅_賽考利克,tuliq kmal qu yaba’ nya’ mha: “Temu’! nyux mak...,他的父親哀痛對膽小的兒子說：「帖姆！我這樣把自己的軀體給了你，,傳說故事精選篇
137191,Atayal,泰雅_賽考利克,"pyang maku’ mha qaniy gi klpngan su’ iyal,",我是故意這樣做的，,傳說故事精選篇
137192,Atayal,泰雅_賽考利克,teta’ ini’ kngungu’ qu laqi’ ta’ mha saku’.”,目的是希望你不要太膽小了，從此成為人中的男人」。,傳說故事精選篇
137193,Atayal,泰雅_賽考利克,"ke’ kasa lga, mskyut qu yaba’ nya’ la.",沒多久父親就與世長辭了。,傳說故事精選篇


# 02-16 更新:只要泰雅語就好, 將 Rau(1992) 資料加進來

In [77]:
import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220106.pkl', compression="gzip")

df_atayal = df[df['Lang_En'] == 'Atayal']
print(df_atayal.shape)

r = df_atayal.copy()

(11868, 5)


,Lang_En,Lang_Ch,Ab,Ch,From
65376,Atayal,泰雅_賽考利克,abas maku' kmmuy qutux kinutan syam qani. ana ...,枉費我把這塊肉捲藏於手掌之中，又沒多少！,詞典
65377,Atayal,泰雅_賽考利克,nyux t'aring mhuq abaw la.,葉子開始掉落了。,詞典
65378,Atayal,泰雅_賽考利克,hata' cin'abaw abaw na trwi' kira' ma'?,待會兒我們去採桑葉好嗎？,詞典
65379,Atayal,泰雅_賽考利克,ungat abura' nya' otobay maku' la!,我的機車沒有油了！,詞典
65380,Atayal,泰雅_賽考利克,"Acung ga, mrhuw pqwasan biru'.",Acung是校長。,詞典
...,...,...,...,...,...
137190,Atayal,泰雅_賽考利克,tuliq kmal qu yaba’ nya’ mha: “Temu’! nyux mak...,他的父親哀痛對膽小的兒子說：「帖姆！我這樣把自己的軀體給了你，,傳說故事精選篇
137191,Atayal,泰雅_賽考利克,"pyang maku’ mha qaniy gi klpngan su’ iyal,",我是故意這樣做的，,傳說故事精選篇
137192,Atayal,泰雅_賽考利克,teta’ ini’ kngungu’ qu laqi’ ta’ mha saku’.”,目的是希望你不要太膽小了，從此成為人中的男人」。,傳說故事精選篇
137193,Atayal,泰雅_賽考利克,"ke’ kasa lga, mskyut qu yaba’ nya’ la.",沒多久父親就與世長辭了。,傳說故事精選篇


In [45]:
rau_1992 = pd.read_excel("data/Rau_1992_text_story.xlsx", sheet_name=None)

In [42]:
rau_1992['01']

,Atayal,English
0,qutux kneril qani uzi ga.,This is also a story about a woman.
1,musa' ma... magaN i... panga' qhoniq ma lru in...,She went to get or carry firewood or maybe she...
2,"ana ga nyaN, smsun squ buli’ uzi ma qu' kneriN...","The woman took a knife with her, which was con..."
3,"ana ga, ini' qbaq ryax i...mromun e... tmutu' ...",She did not notice what time it was. While she...
4,"si ktay nyaN qu ngarux lmru, kyapun nya' qu kn...","all of a sudden, a bear came and grabbed the w..."
5,"nanu' yasa qu, kneriN qani uzi lga, nanak qu n...","As far as the woman was concerned, the bear wa..."
6,"mita’ squ kneriN mga, nanu' si ga', psliqan nq...","When he saw the woman, he tore her clothes. An..."
7,"nanu' spanga' nya' lru, rasun nya' squ ska' hl...",He carried her on his back and took her to the...
8,"ktan lwiy ga, rasun nya' qwang bih na hlahuy m...","Lo and behold, he took her to the deep forest."
9,"rasun nya' qwang bih hlahuy lsa lwiy ga, nyux ...","After he took her to the deep forest, there wa..."


In [46]:
for sheet in rau_1992:
    rau_1992[sheet] = rau_1992[sheet].dropna()
    print("shape of sheet", sheet, rau_1992[sheet].shape)
    rau_1992[sheet] = rau_1992[sheet].rename(columns={"Atayal": "Ab", "English": "Ch"})
    rau_1992[sheet]['Lang_En'] = 'Atayal'
    rau_1992[sheet]['Lang_Ch'] = '泰雅_賽考利克'
    rau_1992[sheet]['From'] = 'Rau (1992)'
    
    r = pd.concat([r, rau_1992[sheet]], ignore_index=True)
    print(r.shape)
    print('======')

shape of sheet 01 (55, 2)
(11923, 5)
shape of sheet 02 (49, 2)
(11972, 5)
shape of sheet 04 (61, 2)
(12033, 5)
shape of sheet 06-1 (14, 2)
(12047, 5)
shape of sheet 06-2 (8, 2)
(12055, 5)
shape of sheet 06-3 (17, 2)
(12072, 5)
shape of sheet 06-4 (7, 2)
(12079, 5)
shape of sheet 06-5 (4, 2)
(12083, 5)
shape of sheet 06-6 (4, 2)
(12087, 5)


In [47]:
r.shape

(12087, 5)

In [48]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# 02-17 更新: 整理大崁崁群的word檔

In [18]:
import pathlib
from collections import defaultdict

In [32]:
from docx import Document
from pprint import pprint

def 檢查各檔案的每個句子的tag(path):

    doc = Document(path)

    having_five_tags = False
    after_title_part = False
    after_new_line = False

    all_sentences = []

    for para in doc.paragraphs:

        if not after_title_part:
            if not para.text.startswith("A:") and\
            not para.text.startswith("RA:") and\
            not para.text.startswith("G:") and\
            not para.text.startswith("M:") and\
            not para.text.startswith("RM:"):
                continue

        after_title_part = True

        # 遇到換行
        if len(para.text.strip()) == 0:
            after_new_line = True
            all_sentences.append(dict())

        if not para.text.startswith("A:") and\
        not para.text.startswith("RA:") and\
        not para.text.startswith("G:") and\
        not para.text.startswith("M:") and\
        not para.text.startswith("RM:"):
            continue

        tag = para.text.split(":")[0]
        content = para.text.lstrip(tag + ":").strip()
        if content == 'none':
            content = None
        

        if len(all_sentences) == 0:
#             all_sentences.append(set())
            all_sentences.append(dict())


#         all_sentences[-1].add(tag)
        all_sentences[-1][tag] = content

    
#     pprint(all_sentences)
    return all_sentences
    #     print(tag)

In [63]:
results = pd.DataFrame([{'Ab':None, 'Ch':None}])

for x in pathlib.Path("data/泰雅族大嵙崁群的部落故事").glob("*aligned.docx"):
    print(f"[[[{x}]]]")
    all_sentences = 檢查各檔案的每個句子的tag(x)
    all_sentences = pd.DataFrame(all_sentences)

    for index, row in all_sentences.iterrows():
        ab = ""
        ch = ""
        
        if 'RA' not in row:
            ab = row['A']
        else:
            if row['RA'] is None:
                ab = row['A']
            else:
                ab = row['RA']

        if 'RM' not in row:
            ch = row['M']
        else:
            if row['RM'] is None:
                ch = row['M']
            else:  
                ch = row['RM']
        
        if ab is None or ch is None:
            continue
            
        results = results.append({'Ab':ab, 'Ch':ch, 'Source': x.name[:4] + "_" + str(index)}, ignore_index = True)

[[[data\泰雅族大嵙崁群的部落故事\TM01_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM02_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM03_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM04_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM05_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM06_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM07_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM08_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM10_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM11_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM12_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM14_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM15_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM16_aligned.docx]]]
[[[data\泰雅族大嵙崁群的部落故事\TM17_aligned.docx]]]


In [65]:
from IPython.display import display
pd.set_option('display.max_rows', 1000)

results = results.dropna()
results['From'] = '大嵙崁群故事'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

display(results.dropna())

,Ab,Ch,Source,From,Lang_En,Lang_Ch
1,zywan mrkyas pinzywan na,等不回的愛情,TM01_0,大嵙崁群故事,Atayal,泰雅_賽考利克
2,"musa maku pqzyu qani ga , zyaw wal mhtuw qalan...",我要講的這個，是從Kekoray的部落傳出的事，,TM01_1,大嵙崁群故事,Atayal,泰雅_賽考利克
3,"maki sazing hi qu msyahaw qasa , mlikuy ga Los...",有一對恩愛夫妻，男生是Losing Yagu，女生是Loba Umaw，這對夫妻很相愛，,TM01_2,大嵙崁群故事,Atayal,泰雅_賽考利克
4,"bsiq balay qu pinlgan nha uzi , ana ga ini qay...",他們也在一起很久了，但是沒有生養小孩，,TM01_3,大嵙崁群故事,Atayal,泰雅_賽考利克
5,"maki qutux ryax , kyalun ni mama nya Tanga Wat...",有一天，女方的叔叔Tanga Watan告訴她說：,TM01_4,大嵙崁群故事,Atayal,泰雅_賽考利克
6,""" Loba ! nyux simuw ini qayat laqi qani , siy ...",「Loba ! 至今你們沒有生養兒女，這個女婿就不要了，你們沒有小孩的話，你老了之後要怎麼生活？」,TM01_5,大嵙崁群故事,Atayal,泰雅_賽考利克
7,"yasa ringan zywaw lru maki qutux ryax lga , t-...",這件事發生之後，有一天，這兩人真的分開了。,TM01_6,大嵙崁群故事,Atayal,泰雅_賽考利克
8,"trang maki qsiyap qu ngasal Tanga Watan , kray...",剛好Tanga Watan的家在對面，過橋都是他的耕地，Loba就去跟她叔叔一起住。,TM01_7,大嵙崁群故事,Atayal,泰雅_賽考利克
9,"Losing qani ga, yaqeh balay qslyaq nya , waliy...",Losing的心情很不好，他的心裡想說：「我們又沒有吵架，我的妻子怎麼能離開我？」,TM01_8,大嵙崁群故事,Atayal,泰雅_賽考利克
10,"qutux qutux ryax ga , musa m'ra squ ubah , mus...",每天XXX，眼望著對岸，想著他的妻子Loba，我們要像這樣生活嗎？,TM01_9,大嵙崁群故事,Atayal,泰雅_賽考利克


In [66]:
results[results['Source'] == 'TM01_3']

,Ab,Ch,Source,From,Lang_En,Lang_Ch
4,"bsiq balay qu pinlgan nha uzi , ana ga ini qay...",他們也在一起很久了，但是沒有生養小孩，,TM01_3,大嵙崁群故事,Atayal,泰雅_賽考利克


In [67]:
import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220216.pkl', compression="gzip")

# df_atayal = df[df['Lang_En'] == 'Atayal']
# print(df_atayal.shape)

r = df.append(results, ignore_index=True)
r

,Lang_En,Lang_Ch,Ab,Ch,From,Source
0,Atayal,泰雅_賽考利克,abas maku' kmmuy qutux kinutan syam qani. ana ...,枉費我把這塊肉捲藏於手掌之中，又沒多少！,詞典,NaN
1,Atayal,泰雅_賽考利克,nyux t'aring mhuq abaw la.,葉子開始掉落了。,詞典,NaN
2,Atayal,泰雅_賽考利克,hata' cin'abaw abaw na trwi' kira' ma'?,待會兒我們去採桑葉好嗎？,詞典,NaN
3,Atayal,泰雅_賽考利克,ungat abura' nya' otobay maku' la!,我的機車沒有油了！,詞典,NaN
4,Atayal,泰雅_賽考利克,"Acung ga, mrhuw pqwasan biru'.",Acung是校長。,詞典,NaN
...,...,...,...,...,...,...
12266,Atayal,泰雅_賽考利克,"sraral maki bih qalang ga , ana bnkis ana mrky...",以前在部落裡，不論老少還是男女，所有人都很喜歡咬著煙斗，去工作或是晚上在家中時，大家都會抽煙...,大嵙崁群故事,TM17_9
12267,Atayal,泰雅_賽考利克,"qruzyux qu nquy tutuh na kneril hiya , r’toh q...",女人的nquy是長的，男人的是短的，就是他們拿來吸菸的，然後，在晚上的山上，很多飛蟲會聚集在...,大嵙崁群故事,TM17_10
12268,Atayal,泰雅_賽考利克,"kruma bnkis ga , qmasan nha qlluy syam qu baba...",有的老人家，會在煙上面醃qlluy豬肉，當菸草冒煙時，菸草帶來的煙真的非常好聞，我每天晚上都...,大嵙崁群故事,TM17_11
12269,Atayal,泰雅_賽考利克,"kbalay qutux tutoh muci yaya ga , ini phmci km...",我媽媽說要做一個煙斗，是不能隨便做的，也相當費時，需要去深入竹林揀選比較老的竹節，btaqu...,大嵙崁群故事,TM17_12


In [68]:
r.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")

# Huang 1994

In [69]:
from docx import Document
from pprint import pprint

import pickle
import pandas as pd

df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220217.pkl', compression="gzip")

In [70]:
doc = Document("data/Huang_1994_text_aligned.docx")
    
results = []

for para in doc.paragraphs:
    print(para.text)
    
    if para.text.startswith("a:"):
        text = para.text.lstrip("a:").strip()
        results.append({"Ab": text})
    elif para.text.startswith("e:"):
        text = para.text.lstrip("e:").strip()
        results[-1]["Ch"] = text

Huang 1994, Ergativity in Atayal, pp. 135–142
Record date: Dec. 1989
Speaker: Ternu Pehu (You Jin-chuan), Wulai
NB: the same story as in Rau 1992



a: qutux knerin qani uzi ga’. 
e: This (story) is also about a woman.

a: musa’ magan panga’ qhuniq ma, lru’ ini’ ga’, ki’a musa’ mqumah squ’ qmayah uzi ga’.
e: It is said that she (the woman) was going to carry wood on her back, or that perhaps she was going to work in the field.

a: ana ga’ nyux smsun squ’ buli’ uzi ma.
e: It is said that the woman also carried a knife with her.

a: yaqu’ knerin qani hiya’ ga’, ini’ qbaq ryax, mromun tmutu’ qhuniq lga’.
e: As for this woman, she had her head lowered cutting trees, without noticing how time passed.

a: si ktay nyan qu’ ngarux lmru’, kyapun nya’ qu’ knerin qani uzi ma.
e: It is said that suddenly a bear came and grabbed this woman.

a: nanu’, nanak qu’ ngarux uzi wah.
e: Well, this bear was special too.

a: mita’ squ’ knerin maga’, nanu’ si ga’, psliqan qu’ lukus knerin, ru’ nanu’ yasa qu’

In [71]:
results = pd.DataFrame(results)
results['From'] = 'Huang 1994'
results['Lang_En'] = 'Atayal'
results['Lang_Ch'] = '泰雅_賽考利克'

In [72]:
df = df.append(results, ignore_index=True)
df
df.to_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")

In [7]:
# Debug

In [9]:
df = pd.read_pickle('./data/Formosan-Mandarin_sent_pairs_20220302.pkl', compression="gzip")